In [1]:
import torch
import re
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import random
import time
import numpy as np
import tiktoken
tiktoken_tokenizer = tiktoken.get_encoding("cl100k_base")
from datetime import datetime
import pytz
import json

%run Azure_QA_utils.ipynb

In [2]:
# Authenticates with the Hugging Face Hub using provided API key

from huggingface_hub import login

api_key = os.getenv('HUGGINGFACE_API_KEY')

login(api_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/drossini/.cache/huggingface/token
Login successful


In [3]:
# Authenticates with the AzureOpenAI using provided API key

from openai import AzureOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")

deployment_name = 'turbo_RD_PROJECTS_DEV'

client = AzureOpenAI(
    api_key=openai_api_key,  
    api_version="2023-07-01-preview",
    azure_endpoint = "https://eastus.api.cognitive.microsoft.com/"
    )

In [4]:
# Loading squad_v2 dataset (downloads if not locally stored)

from datasets import load_dataset

dataset_name = "squad_v2"

local_dataset_dir = "/home/drossini/datasets/squad_v2/"

if not os.path.exists(local_dataset_dir):
    print("Dataset not found. Downloading...")
    dataset = load_dataset(dataset_name, cache_dir=local_dataset_dir)
    print("Dataset downloaded and saved to:", local_dataset_dir)
else:
    print("Dataset found at:", local_dataset_dir)
    dataset = load_dataset('squad_v2', cache_dir=local_dataset_dir)

dataset_train = dataset['train']
# dataset_validation = dataset['validation']

print(f"Training dataset loaded with {len(dataset_train)} examples")
# print(f"Validation dataset loaded with {len(dataset_validation)} examples")

# Filling the empty answers of the training dataset
dataset_train = dataset_train.map(fill_empty_answers)

Dataset found at: /home/drossini/datasets/squad_v2/
Training dataset loaded with 130319 examples


In [1]:
# Loading the model to test the answers with the dataset

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

model_name = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name,  output_hidden_states = True, trust_remote_code=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
# model = model.to(device).half()

# Define the padding to be on the left side
tokenizer.padding_side = "left"

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# # Uncomment if Qwen Model
# if tokenizer.pad_token is None:
#     tokenizer.pad_token_id = tokenizer.eod_id
#     tokenizer.pad_token = '<|endoftext|>'

# config = AutoConfig.from_pretrained(model_name)
# config.torch_dtype = "float16"

/home/drossini/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/drossini/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [6]:
# Launch the experience

experience_timestamps = get_qa_from_squad_v2_textGen(model=model, 
                                                     model_name=model_name,
                                                     tokenizer=tokenizer,
                                                     dataset=dataset_train, 
                                                     dataset_name=dataset_name,
                                                     device=device, 
                                                     deployment_name=deployment_name,
                                                     client=client,
                                                     n_answers=1200, 
                                                     max_tokens=2048, 
                                                     temperature=0.1,
                                                     shuffle_dataset=True,
                                                     batch_size=12)

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


assistant
The title ofrex teutonicum "King of the Germans" is first used in the late 11th century, by the chancery of Pope Gregory VII, to describe the future Holy Roman Emperor of the German Nation Henry IV.


Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


assistant
Camouflaging.
assistant
immunoassays are not banned from being used for diagnostic purposes.
assistant
The only state-run educational system in Ireland prior to comprehensive schools was the vocational school system.
assistant
On 10 January 1970.
assistant
235 marine organisms live at the South Pole all year.
assistant
Central Government of India
assistant
The schools agreed to contribute another £3m to a new charitable educational fund.
assistant
76
assistant
Interracial relations between Indigenous Americans and African Americans are a part of American history that has been neglected.
assistant
Stakeholders use various community stakeholders to implement a marketing strategy, including:

  * Nonprofit organizations
  * Government agencies
  * Community leaders
  * Local businesses
  *志愿者 groups

These stakeholders play a crucial role in identifying the target audience, developing messaging, and executing marketing campaigns. By leveraging their expertise and resources, thes

Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


assistant
The Chinese wanted to receive silver as payment for their opium trade.
assistant
Football is highly popular among Nepali youth.
assistant
In some cases, the federal government may develop a comprehensive scheme that preempts virtually all state law, while in others, like family law, a relatively small number of federal statutes interact with a much larger body of state law.
assistant
Red is associated with Christianity.
assistant
The Rev. Michael J. Shea, a 1904 graduate, wrote the music for "Notre Dame Victory March".
assistant
The torch visited six continents during its journey.
assistant
Birds
assistant
Captain Britain
assistant
August's usual temperature is 66.1 °F (18.9 °C).
assistant
Sir Dave Richards was appointed in April 1999.
assistant
She was 26 years old.
assistant
The Pireto river runs through pallermo.
Batch 960 - 972 generation time: 0:0:0:1.6


Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


assistant
4,500 grants.
assistant
12,000-14,000
assistant
5 August 1920.
assistant
Catholic
assistant
The subgroup of phytochemicals that makes up the majority of the body's antioxidants is polyphenols.
assistant
There are two satellite constellations in the BeiDou Navigation Satellite System.
assistant
The game products represented on game products included bombast artwork, exaggerated graphics, and large variations in graphics, sound, and general quality between the versions.
assistant
Durham Cathedral was abandoned in 1133.
assistant
The more strict program that the USAF adopted in 2010 called the AffT.
assistant
The Labour Party has twenty MEPs in the European Parliament.
assistant
57%
assistant
Szerbs of Croatia are mostly descendants of the Grenzers.
Batch 972 - 984 generation time: 0:0:0:1.2


OutOfMemoryError: CUDA out of memory. Tried to allocate 52.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 39.12 MiB is free. Process 3822697 has 15.73 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 1.29 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)